In [1]:
pip install PySimpleGUI

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import PySimpleGUI as sg
import pandas as pd
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier as rf
import numpy as np
from sklearn.impute import SimpleImputer

# Read data from CSV
excel = pd.read_csv(r"C:\Users\JOYTI\Downloads\sensor.csv")

# Preprocessing
le = preprocessing.LabelEncoder()
pump = le.fit_transform(list(excel["machine_status"]))
SENSOR_00 = list(excel["sensor_00"])
SENSOR_01 = list(excel["sensor_01"])
SENSOR_02 = list(excel["sensor_02"])
SENSOR_03 = list(excel["sensor_03"])
SENSOR_04 = list(excel["sensor_04"])
SENSOR_05 = list(excel["sensor_05"])

features = np.array([SENSOR_00, SENSOR_01, SENSOR_02, SENSOR_03, SENSOR_04, SENSOR_05]).transpose()
features[np.isnan(features)] = 0
imputer = SimpleImputer(strategy='mean')
features = imputer.fit_transform(features)
nan_rows = np.isnan(features).any(axis=1)
features = features[~nan_rows]
pump = pump[~nan_rows]

# Model training
model = rf(n_estimators=3)
model.fit(features, pump)

# Suggestion based on output result
suggestions = {
    'BROKEN': "Please check the water pump for any mechanical issues and perform necessary repairs.",
    'NORMAL': "The water pump is operating within normal parameters.",
    'RECOVERING': "The water pump is currently in a recovery phase. Keep monitoring its performance."
}

# Predictive maintenance methods based on status
maintenance_methods = {
    'BROKEN': "Perform comprehensive inspection and maintenance to identify and rectify the root cause of the failure.",
    'NORMAL': "Continue with regular maintenance checks and monitoring to ensure optimal performance.",
    'RECOVERING': "Implement preventive measures and schedule maintenance to prevent further deterioration and ensure recovery."
}

# GUI Layout
sg.theme('DarkBlue17')

layout = [
    [sg.Text('WATER PUMP FAILURE PREDICTION', font=("Helvetica", 25), text_color='orange', justification='center')],
    [sg.Text('SENSOR RANGES:', font=("Helvetica", 15), justification='left')],
    [sg.Text('SENSOR_00 Range: 0-2'), sg.Text('SENSOR_01 Range: 0-56')],
    [sg.Text('SENSOR_02 Range: 33-56'), sg.Text('SENSOR_03 Range: 31-48')],
    [sg.Text('SENSOR_04 Range: 2-800'), sg.Text('SENSOR_05 Range: 0-99')],
    [sg.Text('_' * 80)],
    [sg.Text('Enter Sensor Readings', font=("Helvetica", 15), justification='center')],
    [sg.Text('SENSOR_00:', size=(12, 1), text_color='white'), sg.Input(font=("Helvetica", 20), size=(20, 1), key='-SENSOR_00-')],
    [sg.Text('SENSOR_01:', size=(12, 1), text_color='white'), sg.Input(font=("Helvetica", 20), size=(20, 1), key='-SENSOR_01-')],
    [sg.Text('SENSOR_02:', size=(12, 1), text_color='white'), sg.Input(font=("Helvetica", 20), size=(20, 1), key='-SENSOR_02-')],
    [sg.Text('SENSOR_03:', size=(12, 1), text_color='white'), sg.Input(font=("Helvetica", 20), size=(20, 1), key='-SENSOR_03-')],
    [sg.Text('SENSOR_04:', size=(12, 1), text_color='white'), sg.Input(font=("Helvetica", 20), size=(20, 1), key='-SENSOR_04-')],
    [sg.Text('SENSOR_05:', size=(12, 1), text_color='white'), sg.Input(font=("Helvetica", 20), size=(20, 1), key='-SENSOR_05-')],
    [sg.Text('', size=(40, 1), font=("Helvetica", 15), text_color='red', key='-ERROR-')],
    [sg.Text(size=(50, 1), font=("Helvetica", 20), text_color='orange', key='-OUTPUT1-', justification='center')],
    [sg.Text(size=(50, 3), font=("Helvetica", 15), text_color='white', key='-SUGGESTION-', justification='center')],
    [sg.Text(size=(50, 3), font=("Helvetica", 15), text_color='white', key='-MAINTENANCE_METHOD-', justification='center')],
    [sg.Button('Submit', font=("Helvetica", 20), button_color=('white', 'green')), sg.Button('Quit', font=("Helvetica", 20), button_color=('white', 'red'))]
]

# Create the window
window = sg.Window('Water Pump Failure Prediction', layout, element_justification='c')

# Event loop
while True:
    event, values = window.read()
    if event == sg.WINDOW_CLOSED or event == 'Quit':
        break

    if event == 'Submit':
        # Validation
        error_msg = ""
        for i in range(6):
            sensor_value = values[f'-SENSOR_{i:02}-']
            if not sensor_value:
                error_msg = f"SENSOR_{i:02} value is required."
                break
            try:
                sensor_value = float(sensor_value)
                if i == 0 and not (0 <= sensor_value <= 2):
                    error_msg = "SENSOR_00 value should be between 0 and 2."
                    break
                elif i == 1 and not (0 <= sensor_value <= 56):
                    error_msg = "SENSOR_01 value should be between 0 and 56."
                    break
                elif i == 2 and not (33 <= sensor_value <= 56):
                    error_msg = "SENSOR_02 value should be between 33 and 56."
                    break
                elif i == 3 and not (31 <= sensor_value <= 48):
                    error_msg = "SENSOR_03 value should be between 31 and 48."
                    break
                elif i == 4 and not (2 <= sensor_value <= 800):
                    error_msg = "SENSOR_04 value should be between 2 and 800."
                    break
                elif i == 5 and not (0 <= sensor_value <= 99):
                    error_msg = "SENSOR_05 value should be between 0 and 99."
                    break
            except ValueError:
                error_msg = f"Invalid value for SENSOR_{i:02}."

        if error_msg:
            window['-ERROR-'].update(error_msg)
            window['-SUGGESTION-'].update('')
            window['-MAINTENANCE_METHOD-'].update('')
        else:
            window['-ERROR-'].update('')
            # Get sensor readings from input fields
            sensor_readings = [float(values[f'-SENSOR_{i:02}-']) for i in range(6)]

            # Predict
            prediction = model.predict([sensor_readings])[0]

            # Interpret prediction
            status = 'UNKNOWN'
            if prediction == 0:
                status = 'BROKEN'
            elif prediction == 1:
                status = 'NORMAL'
            elif prediction == 2:
                status = 'RECOVERING'

            # Update output, suggestion, and maintenance method
            window['-OUTPUT1-'].update(f'Status: {status}', text_color='white')
            window['-SUGGESTION-'].update(suggestions.get(status, ''), text_color='white')
            window['-MAINTENANCE_METHOD-'].update(maintenance_methods.get(status, ''), text_color='white')

# Close the window
window.close()
